# Imports y configuración inicial

In [ ]:
import os
import pandas as pd
import geopandas as gpd # Point para crear geometrías de puntos
import numpy as np
from shapely.geometry import Point # Point para crear geometrías de puntos
from shapely.ops import unary_union, voronoi_diagram # unary_union: Sirve para fusionar muchas geometrías en una sola
from tqdm.auto import tqdm # Crea barras de progreso
import osmnx as ox
import folium
from folium.features import GeoJsonTooltip
from IPython.display import display # Fuerza a Jupyter a mostrar objetos
import warnings # Ocultar los warnings no críticos para mantener la salida limpia
import unicodedata # obtener información detallada sobre cualquier carácter Unicode (como su nombre, clase bidireccional,etc.)
from azure.storage.blob import BlobServiceClient # Conexión con Azure
from pyspark.sql import SparkSession # pyspark
from pyspark.sql.functions import col, lit

# Configuración de librerías
warnings.filterwarnings("ignore") # Ocultar los warnings no críticos para mantener la salida limpia
ox.settings.log_console = False # No llenar la consola de mensajes de registro
ox.settings.use_cache = True # OSMnx para guardar en memoria caché las descargas de mapas la segunda vez es más rápido

In [ ]:
# Azure
connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
access_key = os.getenv("AZURE_STORAGE_KEY")

blob_service_client = BlobServiceClient.from_connection_string(connection_string) # Crea el "puente" para poder entrar a los archivos guardados en Azure

# Spark
spark = (
    SparkSession.builder
    .appName("MicrozonasColombia") # Le pone nombre a esta tarea
    .config("spark.driver.maxResultSize", "4g") # Reserva 4GB de memoria para resultados
    .getOrCreate() # Enciende el motor de Spark
) 

# Da la llave de Azure a Spark para que pueda leer los archivos directamente desde el "Data Lake" (el almacén de datos masivo)
spark.sparkContext._jsc.hadoopConfiguration().set(
    "fs.azure.account.key.kodatasciencedatalake.dfs.core.windows.net",
    access_key
)

In [ ]:
# CRS
EPSG_WGS84 = "EPSG:4326" # Es el sistema de Latitud/Longitud (el que usa el GPS y Google Maps)
EPSG_M = "EPSG:3857" # Si intentas calcular áreas o distancias usando grados (4326), el cálculo saldrá mal. Para medir metros cuadrados o distancias reales, debes convertir los datos a este sistema (3857)

# Rangos de clasificación de microzonas
RANGOS = {
    "chica": (0, 50),
    "mediana": (50, 100),
    "grande": (100, 200),
    "separar": (200, 1e9)
}

# Configuración Data Lake (si para guardar en Azure)
CONTAINER_DATALAKE = "dl-plata"
DATALAKE_NAME = "kodatasciencedatalake"
PROYECTO = "microzonas"
CATEGORIA = "microzonas_colombia"

# Recopilación y filtrado de datos

## Limpieza y Validación

In [ ]:
def limpiar_texto(texto):
    """Limpieza  de texto: elimina tildes, espacios extras, caracteres especiales."""
    if pd.isna(texto): # protege la función si le llega un valor vacío
        return ""

    # Convierte el dato a texto (por si llega un número) y elimina los espacios en blanco al principio y al final.
    texto = str(texto).strip()
    
    # Eliminar tildes
    texto = ''.join(
        c for c in unicodedata.normalize('NFD', texto) # Descompone las letras con tildes. Por ejemplo, separa la 'á' en 'a' + '´'
        if unicodedata.category(c) != 'Mn' # Elimina la parte del acento ('´') y deja solo la letra base
    )
    
    # Convertir a mayúsculas
    texto = texto.upper()
    
    # Eliminar caracteres especiales comunes
    texto = texto.replace('D.C.', '').replace('DC', '').replace('.', '') # Quita puntos, "D.C." y espacios extra
    texto = texto.replace(',', '').replace('-', ' ').replace('_', ' ')
    
    # Normalizar espacios múltiples
    texto = ' '.join(texto.split()) # Divide el texto en palabras y las vuelve a unir con un solo espacio. Esto elimina los dobles o triples espacios que quedan en medio de las palabras
    
    return texto


def validar_input_departamento(nombre, df_localidades):
    """
    Valida y limpia el input del departamento antes de procesarlo.
    Retorna el nombre validado o lanza error con sugerencias.
    """
    if not nombre or not isinstance(nombre, str): # evita que la función se rompa si encuentra NA
        raise ValueError("El nombre del departamento no puede estar vacío")
    
    # Limpiar input
    nombre_limpio = limpiar_texto(nombre)
    
    if not nombre_limpio:
        raise ValueError("El nombre del departamento no es válido después de la limpieza")
    
    # Limpiar nombres del catálogo
    df_localidades['DEPARTAMENTO_LIMPIO'] = df_localidades['DEPARTAMENTO'].apply(limpiar_texto)
    departamentos_disponibles = df_localidades['DEPARTAMENTO_LIMPIO'].unique()
    
    # Buscar coincidencia exacta
    if nombre_limpio in departamentos_disponibles:
        # Retornar el nombre original del catálogo
        return df_localidades[df_localidades['DEPARTAMENTO_LIMPIO'] == nombre_limpio]['DEPARTAMENTO'].iloc[0]
    
    # Buscar coincidencia parcial
    similares = [d for d in departamentos_disponibles if nombre_limpio in d or d in nombre_limpio] # Si no hubo coincidencia exacta, busca si el texto del usuario está contenido dentro de algún nombre de la lista (búsqueda parcial)
    
    if similares: # 
        print(f"'{nombre}' no encontrado exactamente")
        print(f"¿Quisiste decir alguno de estos?")
        for sim in similares[:5]: # Si encuentra parecidos, imprime una lista de sugerencias
            original = df_localidades[df_localidades['DEPARTAMENTO_LIMPIO'] == sim]['DEPARTAMENTO'].iloc[0]
            print(f"      - {original}")
    
    raise ValueError(f"Departamento no válido: '{nombre}'") # detiene el código con un error


def validar_input_municipio(nombre, departamento_validado, df_localidades):
    """
    Valida y limpia el input del municipio antes de procesarlo.
    Retorna el nombre validado o lanza error con sugerencias.
    """
    if not nombre or not isinstance(nombre, str):
        raise ValueError("El nombre del municipio no puede estar vacío")
    
    # Limpiar input
    nombre_limpio = limpiar_texto(nombre)
    
    if not nombre_limpio:
        raise ValueError("El nombre del municipio no es válido después de la limpieza")
    
    # Filtrar municipios del departamento
    df_mun = df_localidades[df_localidades['DEPARTAMENTO'] == departamento_validado].copy()
    df_mun['MUNICIPIO_LIMPIO'] = df_mun['MUNICIPIO'].apply(limpiar_texto)
    municipios_disponibles = df_mun['MUNICIPIO_LIMPIO'].unique()
    
    # Buscar coincidencia exacta
    if nombre_limpio in municipios_disponibles:
        return df_mun[df_mun['MUNICIPIO_LIMPIO'] == nombre_limpio]['MUNICIPIO'].iloc[0]
    
    # Buscar coincidencia parcial
    similares = [m for m in municipios_disponibles if nombre_limpio in m or m in nombre_limpio]
    
    if similares:
        print(f"'{nombre}' no encontrado exactamente en {departamento_validado}")
        print(f"¿Quisiste decir alguno de estos?")
        for sim in similares[:5]:
            original = df_mun[df_mun['MUNICIPIO_LIMPIO'] == sim]['MUNICIPIO'].iloc[0]
            print(f"      - {original}")
    
    raise ValueError(f"Municipio no válido: '{nombre}' en {departamento_validado}")

## Interfaz de Usuario

In [ ]:
# Rutas de datos
RUTA_SECTORES_URBANOS = "abfss://dl-bronce@kodatasciencedatalake.dfs.core.windows.net/GEOESTADISTICO_COLOMBIA/DIVISION_POLITICA/SECTORES_URBANOS"
RUTA_DEPARTAMENTOS_POLITICO = "abfss://dl-bronce@kodatasciencedatalake.dfs.core.windows.net/GEOESTADISTICO_COLOMBIA/DIVISION_POLITICA/DEPARTAMENTOS"
RUTA_LOCALIDADES = "abfss://dl-bronce@kodatasciencedatalake.dfs.core.windows.net/GEOESTADISTICO_COLOMBIA/DIVISION_POLITICA/CATALOGO_LOCALIDADES"

# Cargar catálogo de localidades
try:
    # Leemos con Spark
    df_localidades_spark = spark.read.parquet(RUTA_LOCALIDADES)
    
    # Convertimos a Pandas para poder usar lógica de validación e inputs interactivos
    #    (Solo hacemos esto porque el catálogo es 'pequeño' y cabe en memoria)
    df_localidades = df_localidades_spark.toPandas()
    
    # Normalización básica de columnas (opcional, asegura que sean strings)
    df_localidades['DEPARTAMENTO'] = df_localidades['DEPARTAMENTO'].astype(str)
    df_localidades['MUNICIPIO'] = df_localidades['MUNICIPIO'].astype(str)

except Exception as e:
    print(f"Error crítico cargando catálogo desde Azure: {e}")
    spark.stop()
    raise

# Solicitar tipo de análisis
tipo_analisis = input("\n¿Desea analizar por DEPARTAMENTO completo o por MUNICIPIO específico? (departamento/municipio): ").strip().lower() # quita espacios extra y lo convierte a minúsculas el input

# Manejo de errores
if tipo_analisis not in ["departamento", "municipio"]:
    print("Opción no válida. Se usará 'municipio' por defecto.")
    tipo_analisis = "municipio"

# Solicitar departamento
departamentos_disponibles = df_localidades['DEPARTAMENTO'].unique()
print(f"\nDepartamentos disponibles: {len(departamentos_disponibles)}")
print(f"Ejemplos: {departamentos_disponibles[:5].tolist()}")

departamento_input = input("\nIngrese el nombre del DEPARTAMENTO: ").strip()

# VALIDAR INPUT ANTES DE PROCESAR
try:
    departamento_nombre = validar_input_departamento(departamento_input, df_localidades) # llama a la función validar_input_departamento, le pasa lo que escribió el usuario (_input) y recibe el nombre oficial correcto (_nombre)
    print(f"Departamento validado: {departamento_nombre}")
except ValueError as e: # Si la función de validación no encontró nada (ni exacto ni parecido), captura el error
    print(f"\nError de validación: {e}")
    raise # Detiene el programa

# Variables
municipio_nombre = None # Inicializa la variable vacía. Si el análisis es por departamento completo, esta variable se quedará así (None)
analizar_departamento_completo = (tipo_analisis == "departamento")

if tipo_analisis == "municipio":
    # Filtrar municipios de ese departamento
    municipios_dpto = df_localidades[df_localidades['DEPARTAMENTO'] == departamento_nombre]['MUNICIPIO'].unique()
    
    print(f"\nMunicipios disponibles en {departamento_nombre}: {len(municipios_dpto)}")
    print(f"Ejemplos: {municipios_dpto[:10].tolist()}")
    
    municipio_input = input(f"\nIngrese el nombre del MUNICIPIO: ").strip()
    
    # Validar
    try:
        municipio_nombre = validar_input_municipio(municipio_input, departamento_nombre, df_localidades)
        print(f"Municipio validado: {municipio_nombre}")
    except ValueError as e:
        print(f"\nError de validación: {e}")
        raise
    
    print(f"\nConfiguración: Análisis de municipio específico")
    print(f"Departamento: {departamento_nombre}")
    print(f"Municipio: {municipio_nombre}")
else:
    print(f"\nConfiguración: Análisis de departamento completo")
    print(f"Departamento: {departamento_nombre}")
    print(f"Se procesarán TODOS los municipios del departamento")

## Obtención del resto de datos

In [ ]:
# CARGAR GEOMETRÍA DEL DEPARTAMENTO
try:
    df_depto_geo = spark.read.parquet(RUTA_DEPARTAMENTOS_POLITICO)
    
except Exception as e:
    print(f"No se pudo cargar la geometría del departamento: {e}")

# CARGAR SECTORES URBANOS
try:
    df_sectores = spark.read.parquet(RUTA_SECTORES_URBANOS)

except Exception as e:
    print(f"Error cargando sectores: {e}")

# CARGAR ATRACTORES (POIs)
def obtener_ruta_pois_archivo(dpto_nombre, base_path):
    nombre = dpto_nombre.lower().strip()
    nombre = nombre.replace(",", "").replace(" ", "_")
    return f"{base_path}/pois_{nombre}.csv"

RUTA_BASE_ATRACTORES = "abfss://dl-bronce@kodatasciencedatalake.dfs.core.windows.net/GEOESTADISTICO_COLOMBIA/ATRACTORES"
ruta_archivo_pois = obtener_ruta_pois_archivo(departamento_nombre, RUTA_BASE_ATRACTORES)

try:
    df_pois = spark.read.parquet(ruta_archivo_pois)

except Exception as e:
    print(f"No se pudieron cargar POIs: {e}")

## visualizacion data

In [ ]:
df_localidades

In [ ]:
df_depto_geo_pd = df_depto_geo.toPandas()

df_depto_geo_pd['geometry'] = gpd.GeoSeries.from_wkt(df_depto_geo_pd['geometry'])
gdf_dpto_geo = gpd.GeoDataFrame(df_depto_geo_pd, geometry='geometry', crs=EPSG_WGS84)  # ← Agregar crs

gdf_dpto_geo.head()

In [ ]:
df_sectores_pd = df_sectores.toPandas()

df_sectores_pd['geometry'] = gpd.GeoSeries.from_wkt(df_sectores_pd['geometry'])
gdf_sectores = gpd.GeoDataFrame(df_sectores_pd, geometry='geometry', crs=EPSG_WGS84)  # ← Agregar crs

gdf_sectores.head()

In [ ]:
df_pois_pd = df_pois.toPandas()
df_pois_pd.head()

## Vinculación Geoespacial

In [ ]:
def obtener_codigo_departamento(departamento_nombre_validado, gdf_dpto_geo_input):
    """
    Obtiene el código del departamento desde TODO_COLOMBIA_2024_DPTO_POLITICO.
    NOTA: departamento_nombre_validado ya viene limpio y validado.
    """
    print(f"\nBuscando código para departamento: {departamento_nombre_validado}")
    
    try:
        # Cargar shapefile de departamentos
        gdf_dptos = gdf_dpto_geo_input.copy()
        print(f"   {len(gdf_dptos)} departamentos cargados")
        
        # Limpiar nombres del shapefile
        gdf_dptos['dpto_cnmbr_limpio'] = gdf_dptos['dpto_cnmbr'].apply(limpiar_texto)
        nombre_buscar = limpiar_texto(departamento_nombre_validado)
        
        
        # Buscar coincidencia
        gdf_dpto = gdf_dptos[gdf_dptos['dpto_cnmbr_limpio'] == nombre_buscar].copy()
        
        # Si no encuentra, buscar parcial
        if gdf_dpto.empty:
            print(f"Búsqueda exacta sin resultados, intentando búsqueda parcial...")
            gdf_dpto = gdf_dptos[
                gdf_dptos['dpto_cnmbr_limpio'].str.contains(nombre_buscar, na=False) # Busca si el nombre está contenido dentro de alguno del archivo si escribiste "Santander", encontrará "Norte de Santander" y "Santander"
            ].copy()
        
        # Si aún está vacío, mostrar opciones
        if gdf_dpto.empty:
            print(f"\nNo se encontró '{departamento_nombre_validado}' en el shapefile")
            print(f"Departamentos disponibles en el shapefile:")
            for idx, row in gdf_dptos[['dpto_cnmbr', 'dpto_ccdgo']].head(10).iterrows(): # Ayuda al usuario. Imprime los primeros 10 nombres que SÍ existen en el archivo para que veas por qué falló la búsqueda
                print(f"      - {row['dpto_cnmbr']} (código: {row['dpto_ccdgo']})")
            raise ValueError(f"No se encontró el departamento en el shapefile")
        
        # Si hay múltiples, tomar el primero
        if len(gdf_dpto) > 1: # Si la búsqueda trajo más de un resultado
            print(f"Se encontraron {len(gdf_dpto)} coincidencias, usando la primera")
        
        # Obtener código
        dpto_codigo = gdf_dpto.iloc[0]['dpto_ccdgo']
        dpto_nombre_real = gdf_dpto.iloc[0]['dpto_cnmbr']
        
        print(f"Departamento encontrado en shapefile:")
        print(f"      Nombre: {dpto_nombre_real}")
        print(f"      Código: {dpto_codigo}")
        
        return dpto_codigo, gdf_dpto.to_crs(EPSG_WGS84) # El mapa de ese departamento convertido a coordenadas Latitud/Longitud (EPSG:4326)
        
    except Exception as e:
        print(f"Error: {e}")
        raise

## Función de Filtrado

In [ ]:
def filtrar_sectores_por_departamento(dpto_codigo, gdf_sectores_input, 
                                     municipio_nombre_validado, df_localidades):
    """
    Filtra sectores urbanos por departamento (y opcionalmente por municipio).
    """
    
    print(f"\nFILTRANDO SECTORES URBANOS...")
    print(f"Departamento código: {dpto_codigo}")
    if municipio_nombre_validado: # dar el municipio en caso de haber elegido por municipio
        print(f"   Municipio: {municipio_nombre_validado}")
    
    try:
        # Cargar sectores urbanos
        gdf_sectores = gdf_sectores_input.copy()
        print(f"   Total sectores Colombia: {len(gdf_sectores)}")
        
        # Asegurar CRS
        gdf_sectores = gdf_sectores.to_crs(EPSG_WGS84)
        
        # Filtrar por departamento usando 'dpto_ccdgo' (columna real)
        if 'dpto_ccdgo' not in gdf_sectores.columns:
            raise ValueError("Columna 'dpto_ccdgo' no encontrada en sectores urbanos")
        
        gdf_filtrado = gdf_sectores[gdf_sectores['dpto_ccdgo'].astype(str) == str(dpto_codigo)].copy()
        print(f"{len(gdf_filtrado)} sectores del departamento")
        
        if gdf_filtrado.empty:
            raise ValueError("No se encontraron sectores para el departamento especificado")
        
        # Vincular nombres de municipio desde RUTA_LOCALIDADES
        # Preparar catálogo
        df_loc = df_localidades.copy()
        df_loc['CODIGO_MUNICIPIO'] = df_loc['CODIGO_MUNICIPIO'].astype(str).str.zfill(5) # Asegura que los códigos tengan 5 dígitos rellenando con ceros a la izquierda (ej. convierte "5001" en "05001")
        
        # Vincular usando mpio_ccdgo (columna real en sectores urbanos)
        if 'mpio_ccdgo' in gdf_filtrado.columns: # verifica que exista la columna en los sectores filtrados
            gdf_filtrado['mpio_ccdgo_str'] = gdf_filtrado['mpio_ccdgo'].astype(str).str.zfill(5) # hace lo mismo rellena con ceros para que coincida con el catálogo
            
            # Merge con catálogo
            gdf_filtrado = gdf_filtrado.merge( # Pega la información del catálogo (df_loc) en el mapa (gdf_filtrado) usando el código como llave común.
                df_loc[['CODIGO_MUNICIPIO', 'MUNICIPIO', 'DEPARTAMENTO']],
                left_on='mpio_ccdgo_str',
                right_on='CODIGO_MUNICIPIO',
                how='left'  # how='left': Significa "mantén todos los sectores del mapa, y si encuentras el nombre en el catálogo, pégalo; si no, déjalo vacío".
            )
            
            # Renombrar columna
            gdf_filtrado = gdf_filtrado.rename(columns={'MUNICIPIO': 'nombre_municipio'}) # Cambia el nombre de la columna pegada
            
            print(f"Nombres vinculados: {gdf_filtrado['nombre_municipio'].notna().sum()} sectores")
        else:
            print(f"Columna 'mpio_ccdgo' no encontrada, no se pueden vincular nombres")
            gdf_filtrado['nombre_municipio'] = 'DESCONOCIDO'
        
        # Filtrar por municipio si se especificó
        if municipio_nombre_validado:
            if 'nombre_municipio' in gdf_filtrado.columns:
                gdf_filtrado = gdf_filtrado[
                    gdf_filtrado['nombre_municipio'] == municipio_nombre_validado].copy()
                print(f"{len(gdf_filtrado)} sectores del municipio {municipio_nombre_validado}")
                
                if gdf_filtrado.empty:
                    raise ValueError(f"No se encontraron sectores para el municipio {municipio_nombre_validado}")
        
        # Generar microzona_id
        # Formato: DPTO_MPIO_SECTOR (usando códigos que existen en todos los países)
        print(f"Generando microzona_id...")
        
        componentes = []
        if 'dpto_ccdgo' in gdf_filtrado.columns:
            componentes.append(gdf_filtrado['dpto_ccdgo'].astype(str).str.zfill(2))
        if 'mpio_ccdgo' in gdf_filtrado.columns:
            componentes.append(gdf_filtrado['mpio_ccdgo'].astype(str).str.zfill(3))
        if 'setu_ccdgo' in gdf_filtrado.columns:
            componentes.append(gdf_filtrado['setu_ccdgo'].astype(str).str.zfill(4))
        
        if componentes:
            gdf_filtrado['microzona_id'] = componentes[0]
            for comp in componentes[1:]:
                gdf_filtrado['microzona_id'] = gdf_filtrado['microzona_id'] + '_' + comp
        else:
            # Fallback: usar índice
            gdf_filtrado['microzona_id'] = 'MZ_' + gdf_filtrado.index.astype(str).str.zfill(6) # Si faltan columnas, crea un ID de emergencia usando el número de fila (MZ_000123)
        
        print(f"microzona_id generado para {len(gdf_filtrado)} sectores")
        
        return gdf_filtrado
        
    except Exception as e:
        print(f"Error: {e}")
        raise

## Ingesta y Filtrado de Puntos de Interés

In [ ]:
def cargar_establecimientos_csv(df_pois_pd_input, gdf_sectores):
    """
    Carga establecimientos desde CSV y filtra por el área de los sectores.
    """    
    try:
        # Cargar CSV
        df = df_pois_pd_input
        print(f"{len(df)} establecimientos")
        
        # Validar columnas de coordenadas
        if 'coord_x' not in df.columns or 'coord_y' not in df.columns:
            raise ValueError("Columnas 'coord_x' y 'coord_y' no encontradas en el CSV")
        
        # Eliminar NaN
        df = df.dropna(subset=['coord_x', 'coord_y'])
        
        # Crear GeoDataFrame
        gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['coord_x'], df['coord_y']), crs=EPSG_WGS84)
        
        print(f"{len(gdf)} con coordenadas válidas")
        
        # Filtrar por bbox de sectores
        bbox = gdf_sectores.total_bounds # Calcula la "Caja Delimitadora" (Bounding Box) de tus sectores urbanos (Coordenada mínima X, mínima Y, máxima X, máxima Y)
        gdf_filtrado = gdf.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]].copy() # En lugar de verificar punto por punto si está dentro de un polígono complejo (que es lento), GeoPandas usa el índice espacial cx para decir: "Dame todo lo que esté dentro de este rectángulo"
        
        print(f"{len(gdf_filtrado)} establecimientos en el área de interés")
        
        return gdf_filtrado
        
    except Exception as e:
        print(f"Error: {e}")
        return gpd.GeoDataFrame(columns=['geometry'], crs=EPSG_WGS84)

## Descarga de Red Vial desde OpenStreetMap (OSM)

In [ ]:
def obtener_vialidades(gdf_zona):
    """Descarga vialidades de OSM para el área."""
    print("\nDESCARGANDO VIALIDADES  desde OSM...")
    
    tags_vialidades = {
        "highway": [
            "primary", "secondary", "tertiary",
            "motorway", "trunk",                 # solo las calles por donde circulan carros. Esto excluye automáticamente caminos peatonales (footway), ciclovías (cycleway) o caminos de tierra menores
            "primary_link", "secondary_link",
            "tertiary_link", "motorway_link", "trunk_link"
        ]
    }
    
    try:
        gdf_zona = gdf_zona.to_crs(EPSG_WGS84)
        poly = gdf_zona.unary_union # gdf_zona puede tener 100 sectores (100 polígonos pequeños). Si OSM que busca en cada uno, harás 100 peticiones 
                                    # y será lentísimo unary_union fusiona todos esos en un solo polígono, se hace una sola petición al servidor 
        
        if poly is None or poly.is_empty:
            raise ValueError("Polígono inválido")
        
        # Descargando desde OSM
        gdf_vias = ox.features.features_from_polygon(poly, tags=tags_vialidades) # librería osmnx para conectarse a la API de OpenStreetMap
                                                                                 # envía el polígono grande (poly) y la lista de tipos de calle (tags)
        
        if gdf_vias.empty:
            print("No se encontraron vías") # OSM no devolvió nada
            return gpd.GeoDataFrame(columns=['geometry'], crs=EPSG_WGS84) # devuelve un mapa vacío para no romper el código
        
        gdf_vias = gdf_vias[
            gdf_vias.geometry.type.isin(['LineString', 'MultiLineString']) # OSM marca un semáforo o una rotonda como un "Punto" o un "Polígono". Este filtro elimina todo lo que no sean Líneas
        ].copy()
        gdf_vias = gdf_vias.to_crs(EPSG_WGS84).reset_index(drop=True) # Reorganiza la tabla para que el índice sea limpio (0, 1, 2...)
        
        print(f"{len(gdf_vias)} vialidades descargadas")
        return gdf_vias
        
    except Exception as e:
        print(f"Error: {e}")
        return gpd.GeoDataFrame(columns=['geometry'], crs=EPSG_WGS84)

# Generación de microzonas

## Motor de procesamiento geométrico

In [ ]:
def dividir_sectores_con_vialidades(gdf_sectores, gdf_vialidades, buffer_metros=8):
    """Divide sectores usando vialidades."""
    print(f"\nDIVIDIENDO SECTORES (buffer: {buffer_metros}m)...")
    
    gdf_m = gdf_sectores.to_crs(EPSG_M) # Convierte el mapa de sectores a Metros (EPSG:3857) Las operaciones geométricas como cortar o medir 
                                        # distancias SIEMPRE deben hacerse en metros, no en grados (latitud/longitud), para ser precisas
    
    if gdf_vialidades.empty:
        print("Sin vialidades, usando sectores completos")
        res = gdf_sectores.copy() # Copia los sectores tal cual
        res["sub_microzona_id"] = res["microzona_id"] + "_001" # Les crea un ID nuevo terminando en _001 y retorna
        return res
        
    vias_m = gdf_vialidades.to_crs(EPSG_M) # Convierte también las calles a Metros
    microzonas = [] # Para guardar los pedasos recortados
    
    for _, row in tqdm(gdf_m.iterrows(), total=len(gdf_m), desc="   Cortando"): # Inicia un bucle para procesar sector por sector con barra de progreso
        geom = row.geometry                                                     # Extrae la forma del sector actual
        vias_cercanas = vias_m[vias_m.intersects(geom.buffer(20))]              # En lugar de usar todas las calles de la ciudad para cortar un barrio, selecciona solo las calles que tocan o están a menos de 20 metros de ese barrio
        
        if vias_cercanas.empty:                                                 # Si no hay calles cerca, guarda el sector entero y pasa al siguiente
            microzonas.append(row)
            continue
            
        try:
            cuchilla = vias_cercanas.buffer(buffer_metros).unary_union        # Convierte las líneas de las calles en polígonos gruesos (de 8 metros de ancho, según el parámetro) y los fusiona en una sola forma
            recorte = geom.difference(cuchilla)                               # Resta la "cuchilla" (calles) al "sector" (barrio). Lo que queda son las manzanas aisladas
            
            if recorte.geom_type == "Polygon" and not recorte.is_empty:       # Si el resultado es un solo pedazo y no está vacío, lo guarda
                row.geometry = recorte
                microzonas.append(row)
            elif recorte.geom_type == "MultiPolygon":                         # Si el corte generó varias islas (manzanas separadas)               
                for p in recorte.geoms:                                       # Recorre cada isla          
                    if p.area > 500:                                          # Si el pedazo es muy pequeño (menos de 500m²), lo descarta y si es grande, lo guarda como una nueva microzona
                        new_row = row.copy()
                        new_row.geometry = p
                        microzonas.append(new_row)
            else:
                microzonas.append(row)
        except:
            microzonas.append(row)
            
    gdf_final = gpd.GeoDataFrame(microzonas, crs=EPSG_M).to_crs(EPSG_WGS84) # Convierte todos los pedacitos resultantes de nuevo a Latitud/Longitud
    gdf_final = gdf_final.reset_index(drop=True)
    
    # Generar sub-ID
    gdf_final["sub_id"] = gdf_final.groupby("microzona_id").cumcount() + 1               # Enumera los pedazos (1, 2, 3...) dentro de cada sector original
    gdf_final["sub_microzona_id"] = (
        gdf_final["microzona_id"] + "_S" + gdf_final["sub_id"].astype(str).str.zfill(3)   # Crea el ID final único (ej. 05_001_SEC1_S001)
    )
    
    print(f"{len(gdf_final)} microzonas generadas")
    return gdf_final


def contar_y_clasificar(gdf_micro, gdf_puntos):
    """Cuenta y clasifica."""
    print("Contando establecimientos...")

    # Asegura que ambos mapas (zonas y puntos) estén en el mismo sistema (Lat/Lon)
    gdf_micro = gdf_micro.to_crs(EPSG_WGS84)
    gdf_puntos = gdf_puntos.to_crs(EPSG_WGS84)
    
    # Usar sub_microzona_id si existe, sino microzona_id
    id_col = 'sub_microzona_id' if 'sub_microzona_id' in gdf_micro.columns else 'microzona_id' # Decide qué columna usar como llave. Si ya cortamos con calles, usa sub_microzona_id; si no, usa microzona_id
    
    joined = gpd.sjoin(gdf_puntos, gdf_micro[[id_col, 'geometry']], predicate='within') # Asigna cada punto al polígono que lo contiene.
    
    if joined.empty:                                 # Si ningún punto cayó dentro de ninguna zona
        gdf_micro['total_establecimientos'] = 0
        gdf_micro['categoria'] = 'chica'
        return gdf_micro
        
    conteo = joined.groupby(id_col).size()        # Cuenta cuántos puntos hay por cada ID de zona
    gdf_micro = gdf_micro.set_index(id_col)
    gdf_micro['total_establecimientos'] = conteo
    gdf_micro['total_establecimientos'] = gdf_micro['total_establecimientos'].fillna(0).astype(int) # Las zonas que no tienen tiendas quedan con NaN (nulo); esto las convierte en 0
    gdf_micro = gdf_micro.reset_index()
    
    def clasificar(n): # funcion uinterna, usa el diccionario RANGOS
        for cat, (min_v, max_v) in RANGOS.items():   # Recorre los rangos (0-50, 50-100, etc.) y devuelve la etiqueta correspondiente.
            if min_v <= n < max_v:
                return cat
        return 'separar'
    
    gdf_micro['categoria'] = gdf_micro['total_establecimientos'].apply(clasificar)
    return gdf_micro


def dividir_microzonas_grandes_voronoi(gdf):
    """Divide grandes con Voronoi."""
    print("\nREFINANDO GRANDES...")
    
    gdf = gdf.to_crs(EPSG_M) # Convertir para calcular áreas
    grandes = gdf[gdf['categoria'].isin(['grande', 'separar'])].copy() # Separa las zonas que necesitan dividirse
    otras = gdf[~gdf['categoria'].isin(['grande', 'separar'])].copy() # Separa las zonas que ya están bien
    
    if grandes.empty:
        return gdf.to_crs(EPSG_WGS84)
        
    procesadas = []
    id_col = 'sub_microzona_id' if 'sub_microzona_id' in grandes.columns else 'microzona_id'
    
    for _, row in tqdm(grandes.iterrows(), total=len(grandes), desc="   Voronoi"): # Recorre las zonas grandes tqdm para barra de progreso
        geom = row.geometry
        total = row.total_establecimientos
        n_partes = int(np.ceil(total / 75)) # Calcula en cuántos pedazos dividir
        
        if n_partes < 2:
            procesadas.append(row)
            continue
            
        puntos = [] # Genera puntos aleatorios dentro del polígono para usarlos como "semillas" del corte
        minx, miny, maxx, maxy = geom.bounds
        intentos = 0
        while len(puntos) < n_partes and intentos < n_partes*20:
            p = Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
            if geom.contains(p):
                puntos.append(p)
            intentos += 1
            
        if len(puntos) < 2:
            procesadas.append(row)
            continue
            
        try:
            region = geom.buffer(100)
            vor = voronoi_diagram(unary_union(puntos), envelope=region) # Algoritmo Voronoi. Crea regiones geométricas basándose en la cercanía a los puntos semilla
            sub_id = 1
            for poly in vor.geoms:
                res = geom.intersection(poly) # Recorta el diagrama Voronoi infinito para que encaje exactamente dentro de la forma de la microzona original
                if not res.is_empty and res.area > 50:
                    new_row = row.copy()
                    new_row.geometry = res
                    new_row['total_establecimientos'] = int(total / n_partes) # Divide los puntos equitativamente entre los nuevos pedazos (es una estimación)
                    new_row[id_col] = f"{row[id_col]}_V{sub_id}" # Agrega un sufijo al ID (ej. ..._S001_V1) para indicar que fue dividido por Voronoi
                    procesadas.append(new_row)
                    sub_id += 1
        except:
            procesadas.append(row)
            
    final = pd.concat([gpd.GeoDataFrame(procesadas, crs=EPSG_M), otras], ignore_index=True)
    return final.to_crs(EPSG_WGS84)

def unir_microzonas_chicas(gdf):
    """
    Une microzonas chicas colindantes para formar microzonas medianas.
    OPTIMIZADA: Usa índice espacial (sindex) para búsqueda rápida.
    """
    print("\nUNIENDO MICROZONAS CHICAS...")
    
    # Trabajar en metros es obligatorio para geometría precisa
    gdf = gdf.to_crs(EPSG_M)
    
    # Separar las chicas del resto
    chicas = gdf[gdf['categoria'] == 'chica'].copy().reset_index(drop=True)
    otras = gdf[gdf['categoria'] != 'chica'].copy()
    
    if chicas.empty or len(chicas) < 2:                            # Si no hay zonas chicas, devuelve el mapa original y termina
        print("No hay suficientes microzonas chicas para unir")
        return gdf.to_crs(EPSG_WGS84)
    
    print(f"   {len(chicas)} microzonas chicas para procesar")
    
    # Variables de control
    chicas['processed'] = False # olumna de control (tipo "Checklist") para saber qué zonas ya fusionamos y no repetirlas
    id_col = 'sub_microzona_id' if 'sub_microzona_id' in chicas.columns else 'microzona_id'  # Define qué nombre de columna usar para los IDs
    uniones = []
    grupo_id = 1
    
    for idx, row in tqdm(chicas.iterrows(), total=len(chicas), desc="   Agrupando"):   # Empieza a recorrer cada zona chica, una por una tqdm para barra de progreso
        if chicas.at[idx, 'processed']: # Si esta zona ya fue fusionada en un paso anterior, la salta
            continue
        
        # Iniciar nuevo grupo
        grupo_geoms = [row.geometry]
        grupo_ids = [str(row[id_col])]
        grupo_total = row['total_establecimientos']
        chicas.at[idx, 'processed'] = True # Marcar como usada

        # Define las metas
        target_min = 50
        target_max = 100
        
        # Bucle de crecimiento del grupo
        changed = True
        while changed and grupo_total < target_max: # Mantiene el proceso vivo mientras siga encontrando vecinos válidos y no se haya llenado el grupo
            changed = False
            geom_actual = unary_union(grupo_geoms) # Forma actual del grupo
            
            # --- OPTIMIZACIÓN CON SINDEX ---
            # 1. Filtro Rápido: Buscar solo geometrías cuyo recuadro (bbox) toque el grupo
            posibles_indices = list(chicas.sindex.query(geom_actual, predicate='intersects'))
            
            # 2. Filtro Fino: Revisar uno por uno los candidatos
            for candidato_idx in posibles_indices:
                if candidato_idx == idx or chicas.at[candidato_idx, 'processed']: # Verifica si realmente están pegadas pared con pared
                    continue
                
                row2 = chicas.loc[candidato_idx]
                
                # Verificar contacto real (touches = se tocan, intersects = se solapan)
                if geom_actual.touches(row2.geometry) or geom_actual.intersects(row2.geometry):
                    
                    nuevo_total = grupo_total + row2['total_establecimientos'] # Suma cuántos establecimientos tendríamos si las unimos
                    
                    # Decidir si unimos
                    if nuevo_total <= target_max or grupo_total < target_min: # Solo une si La suma no se pasa de 100 O si el grupo actual es tan pequeño (<50)
                        grupo_geoms.append(row2.geometry) # Si pasa la prueba, agrega la vecina al equipo
                        grupo_ids.append(str(row2[id_col]))
                        grupo_total += row2['total_establecimientos']
                        chicas.at[candidato_idx, 'processed'] = True # Marcar vecina como usada
                        changed = True
                        
                        if grupo_total >= target_min:
                            break # Ya cumplimos la meta, salir a cerrar el grupo
        
        # Guardar el resultado del grupo
        nueva_geom = unary_union(grupo_geoms) # Funde todas las geometrías del equipo en un solo polígono final
        nueva_row = row.copy() # Copiar datos base
        nueva_row.geometry = nueva_geom
        nueva_row['total_establecimientos'] = grupo_total
        # Generar ID compuesto (ej. UNION_1_ID1-ID2)
        ids_cortos = [i.split('_')[-1] for i in grupo_ids[:3]] # Solo tomar la última parte del ID para no hacerlo gigante
        nueva_row[id_col] = f"U{grupo_id}_{'-'.join(ids_cortos)}" 
        
        # Recalcular categoría
        if grupo_total < 50: nueva_row['categoria'] = 'chica'
        elif grupo_total < 100: nueva_row['categoria'] = 'mediana'
        elif grupo_total < 200: nueva_row['categoria'] = 'grande'
        else: nueva_row['categoria'] = 'separar'
        
        uniones.append(nueva_row)
        grupo_id += 1
    
    # Unir resultados
    if uniones:
        gdf_uniones = gpd.GeoDataFrame(uniones, crs=EPSG_M)
        # Traer de vuelta las que quedaron "solteras" (chicas que no se pudieron unir a nadie)
        sobras = chicas[~chicas['processed']].copy()
        
        final = pd.concat([gdf_uniones, sobras, otras], ignore_index=True) # Pega todo de nuevo: (Las uniones nuevas) + (Las sobras chicas) + (Las medianas/grandes originales)
        
        print(f"{len(uniones)} grupos nuevos creados")
        print(f"   Distribución final:")
        print(final['categoria'].value_counts().to_string())
        
        return final.to_crs(EPSG_WGS84)
    else:
        return gdf.to_crs(EPSG_WGS84)

In [ ]:
def pipeline_colombia():
    """Pipeline completo para Colombia."""
    
    # PASO 1: Obtener código de departamento
    dpto_codigo, gdf_dpto = obtener_codigo_departamento(
        departamento_nombre,
        gdf_dpto_geo
    )
    
    # PASO 2: Filtrar sectores (con vinculación de nombres)
    print("\nPASO 2: Filtrando sectores...")
    gdf_sectores_filtrado = filtrar_sectores_por_departamento(
        dpto_codigo,
        gdf_sectores,
        municipio_nombre,
        df_localidades
    )
    
    # PASO 3: Cargar establecimientos
    print("\nPASO 3: Cargando establecimientos...")
    gdf_establecimientos = cargar_establecimientos_csv(df_pois_pd, gdf_sectores_filtrado)

    # --- DEFINICIÓN DEL ESTADO INICIAL (Sectores sin dividir) ---
    print("\nPASO 3.5: Generando Dataset INICIAL (Sectores sin dividir)...")
    microzonas_inicial = contar_y_clasificar(gdf_sectores_filtrado.copy(), gdf_establecimientos)
    
    # PASO 4: Obtener vialidades
    print("\nPASO 4: Obteniendo vialidades...")
    vialidades = obtener_vialidades(gdf_sectores_filtrado)
    
    # PASO 5: Dividir sectores
    print("\nPASO 5: Dividiendo sectores...")
    microzonas_div = dividir_sectores_con_vialidades(gdf_sectores_filtrado, vialidades)
    
    # PASO 6: Contar (Conteo intermedio de las zonas divididas)
    print("\nPASO 6: Contando establecimientos...")
    microzonas_intermedio = contar_y_clasificar(microzonas_div, gdf_establecimientos)
    
    # PASO 7: Refinar grandes
    print("\nPASO 7: Refinando grandes...")
    microzonas_refinadas = dividir_microzonas_grandes_voronoi(microzonas_intermedio.copy())
    
    # PASO 8: Reclasificar después de dividir grandes
    print("\nPASO 8: Reclasificación...")
    microzonas_reclasificadas = contar_y_clasificar(microzonas_refinadas, gdf_establecimientos)
    
    # PASO 9: Unir microzonas chicas colindantes
    print("\nPASO 9: Uniendo microzonas chicas...")
    microzonas_unidas = unir_microzonas_chicas(microzonas_reclasificadas.copy())
    
    # PASO 10: Reclasificar final después de unir
    print("\n PASO 10: Reclasificación final...")
    microzonas_final = contar_y_clasificar(microzonas_unidas, gdf_establecimientos)
    
    # PASO 11: Eliminar vacías
    print("\nPASO 11: Eliminando vacías...")
    antes = len(microzonas_final)
    microzonas_final = microzonas_final[microzonas_final['total_establecimientos'] > 0].copy()
    print(f"   Eliminadas: {antes - len(microzonas_final)}")
    
    # PASO 12: Calcular área
    print("\nPASO 12: Calculando áreas...")
    gdf_temp = microzonas_final.to_crs(EPSG_M)
    microzonas_final['area_km2'] = (gdf_temp.geometry.area / 1_000_000).round(4)

    # PASO 13: LIMPIEZA DE COLUMNAS
    print("\nPASO 13: Generando Datasets Limpios (Inicial y Final)...")
    
    #  A. FUNCION DE REPARACIÓN INTERNA
    def reparar_dataset(gdf_sucio, tipo="final"):
        df = gdf_sucio.copy()
        
        # 1. Definir el ID Correcto
        id_col = 'sub_microzona_id' if 'sub_microzona_id' in df.columns else 'microzona_id'
        
        # 2. Reconstruir códigos desde el ID (El ID nunca se pierde)
        # Formato ID típico: "05_001_XXXX" (Dpto_Mpio_...)
        try:
            # Dpto es la parte 0, Mpio es la parte 1
            df['temp_dpto_cod'] = df[id_col].astype(str).apply(lambda x: x.split('_')[0])
            df['temp_mpio_cod'] = df[id_col].astype(str).apply(lambda x: x.split('_')[1])
            
            # Construimos el código DANE completo (5 dígitos) para buscar el nombre
            df['temp_dane_completo'] = df['temp_dpto_cod'] + df['temp_mpio_cod']
        except Exception as e:
            print(f"Advertencia reparando códigos en dataset {tipo}: {e}")
            # Fallback si el ID no tiene el formato esperado
            df['temp_dpto_cod'] = str(dpto_codigo).zfill(2)
            df['temp_dane_completo'] = "00000"

        # 3. Preparar Diccionario de Nombres (Desde df_localidades que es nuestra fuente de verdad)
        df_loc = df_localidades.copy()
        df_loc['CODIGO_MUNICIPIO'] = df_loc['CODIGO_MUNICIPIO'].astype(str).str.zfill(5)
        mapa_nombres = df_loc.set_index('CODIGO_MUNICIPIO')['MUNICIPIO'].to_dict()
        mapa_dptos = df_loc.set_index('CODIGO_MUNICIPIO')['DEPARTAMENTO'].to_dict()
        
        # 4. Asignar Valores (Reparación)
        df['codigo_departamento'] = df['temp_dpto_cod']
        df['codigo_municipio'] = df['temp_dane_completo']
        
        # Mapear nombres (si no encuentra el código, pone 'DESCONOCIDO')
        df['nombre_municipio'] = df['temp_dane_completo'].map(mapa_nombres).fillna('DESCONOCIDO')
        df['nombre_departamento'] = df['temp_dane_completo'].map(mapa_dptos).fillna('DESCONOCIDO')
        
        # Si el departamento salió desconocido (raro), forzamos el nombre global
        mask_unknown = df['nombre_departamento'] == 'DESCONOCIDO'
        if mask_unknown.any():
            # Intentamos obtener el nombre del departamento input
            try:
                nombre_dpto_global = df_localidades[df_localidades['DEPARTAMENTO_LIMPIO'].str.contains(limpiar_texto(str(departamento_nombre)), na=False)]['DEPARTAMENTO'].iloc[0]
            except:
                nombre_dpto_global = "DESCONOCIDO"
            df.loc[mask_unknown, 'nombre_departamento'] = nombre_dpto_global

        # 5. Calcular Área si falta (para el inicial)
        if 'area_km2' not in df.columns:
            df['area_km2'] = (df.to_crs(EPSG_M).geometry.area / 1_000_000).round(4)
            
        # 6. Selección Final de Columnas
        columnas_finales = {
            id_col: 'id_microzona',
            'geometry': 'geometry',
            'area_km2': 'area_km2',
            'nombre_departamento': 'nombre_departamento',
            'codigo_departamento': 'codigo_departamento',
            'nombre_municipio': 'nombre_municipio',
            'codigo_municipio': 'codigo_municipio',
            'categoria': 'categoria',
            'total_establecimientos': 'total_establecimientos'
        }
        
        # Filtrar y Renombrar
        cols_validas = {k: v for k, v in columnas_finales.items() if k in df.columns}
        df_limpio = df[list(cols_validas.keys())].rename(columns=cols_validas)
        
        return df_limpio.to_crs(EPSG_WGS84)

    # B. APLICAR REPARACIÓN
    print("Reparando dataset FINAL...")
    dataset_final_limpio = reparar_dataset(microzonas_final, tipo="final")
    
    print("Reparando dataset INICIAL...")
    dataset_inicial_limpio = reparar_dataset(microzonas_inicial, tipo="inicial")

    # Reporte final
    print("\nREPORTE FINAL DE MICROZONAS")
    print("\nDistribución por categoría:")
    print(microzonas_final['categoria'].value_counts().to_string())

    return dataset_inicial_limpio, dataset_final_limpio, vialidades, gdf_dpto

In [ ]:
microzonas_inicial, microzonas_final, vialidades, poligono_dpto = pipeline_colombia()

print(f"\n RESULTADOS:")
print(f"   Iniciales: {len(microzonas_inicial)}")
print(f"   Finales: {len(microzonas_final)}")

In [ ]:
microzonas_inicial

In [ ]:
microzonas_final

In [ ]:
def mostrar_mapa_microzonas_bogota(microzonas, vialidades, establecimientos_totales, titulo="Microzonas"):
    """
    Genera mapa interactivo.
    - Muestra microzonas con tooltip completo.
    - Muestra SOLO los establecimientos que caen dentro de los polígonos mostrados.
    """
    print(f"\nGenerando mapa: {titulo}")
    
    # Crear mapa base
    centro = microzonas.to_crs(EPSG_M).geometry.centroid.to_crs(EPSG_WGS84)
    m = folium.Map(
        location=[centro.y.mean(), centro.x.mean()], 
        zoom_start=13, 
        tiles="Cartodb Positron",
        prefer_canvas=True
    )
    
    # 1. Agregar Vialidades (Fondo)
    if not vialidades.empty:
        folium.GeoJson(
            vialidades, name="Vialidades", 
            style_function=lambda x: {"color": "#ff7f00", "weight": 1.5, "opacity": 0.5}
        ).add_to(m)

    # 2. Configurar Tooltips y Colores
    paleta = {"chica": "#92c5de", "mediana": "#f4a582", "grande": "#ca0020", "separar": "#7b3294"}
    
    # Estas son las columnas NUEVAS que definiste en el pipeline
    campos_tooltip = ['id_microzona', 'area_km2', 'nombre_municipio', 'nombre_departamento', 'categoria', 'total_establecimientos']
    # Validar que existan (por seguridad)
    campos_reales = [c for c in campos_tooltip if c in microzonas.columns]
    
    # 3. Dibujar Zonas
    for cat, color in paleta.items():
        capa = microzonas[microzonas["categoria"] == cat].copy()
        if not capa.empty:
            fg = folium.FeatureGroup(name=f"Zonas: {cat.upper()}", show=True)
            
            folium.GeoJson(
                capa,
                style_function=lambda feature, color=color: {
                    "fillColor": color, "color": "black", "weight": 0.5, "fillOpacity": 0.5
                },
                highlight_function=lambda feature: {"fillOpacity": 0.8, "weight": 2, "color": "white"},
                tooltip=folium.GeoJsonTooltip(
                    fields=campos_reales,
                    aliases=[c.replace('_', ' ').title() + ':' for c in campos_reales],
                    style="font-family: Arial; font-size: 12px;"
                )
            ).add_to(fg)
            fg.add_to(m)

    # 4. Dibujar Establecimientos (Filtrados por polígono)
    print("Filtrando y dibujando establecimientos...")
    
    # A. Filtro espacial rápido (Bounding Box)
    minx, miny, maxx, maxy = microzonas.total_bounds
    puntos_cerca = establecimientos_totales.cx[minx:maxx, miny:maxy]
    
    # B. Filtro fino: Solo los que están DENTRO de las geometrías de este dataset
    # Usamos sjoin para quedarnos solo con los puntos que caen en estas microzonas específicas
    puntos_filtrados = gpd.sjoin(puntos_cerca, microzonas[['geometry']], predicate='within')
    
    if not puntos_filtrados.empty:
        fg_puntos = folium.FeatureGroup(name="Establecimientos", show=False)
        
        # Validar columnas del CSV de establecimientos
        col_name = 'name' if 'name' in puntos_filtrados.columns else puntos_filtrados.columns[0]
        col_amenity = 'amenity' if 'amenity' in puntos_filtrados.columns else 'N/A'
        
        folium.GeoJson(
            puntos_filtrados,
            marker=folium.CircleMarker(radius=2, fill=True, fill_opacity=1),
            style_function=lambda x: {"color": "#333333", "fillColor": "#333333", "weight": 0},
            tooltip=folium.GeoJsonTooltip(
                fields=[col_name, col_amenity],
                aliases=['Nombre:', 'Tipo:'],
                style="font-family: Arial; font-size: 11px; font-weight: bold;"
            )
        ).add_to(fg_puntos)
        fg_puntos.add_to(m)

    folium.LayerControl(collapsed=False).add_to(m)
    return m

In [ ]:
# Recargar establecimientos (necesario para pintar los puntos)
print("\nRecargando puntos para el mapa...")
pts_totales = cargar_establecimientos_csv(df_pois_pd, poligono_dpto)

# Mapa Inicial
display(mostrar_mapa_microzonas_bogota(
    microzonas_inicial, vialidades, pts_totales, titulo="INICIAL"
))

# 4. Mapa Final
display(mostrar_mapa_microzonas_bogota(
    microzonas_final, vialidades, pts_totales, titulo="FINAL"
))